In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import warnings
import re
import seaborn as sns
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore')
pd.set_option('display.max_colwidth', None)

# Data Dictionary

In [4]:
data_path = "/content/drive/MyDrive/anna-paula-goncalves/datasets/palavras-dicionario-de-libras.js"
df = pd.read_json(data_path)
df = df.astype({'exemplo':str, 'libras':str}).rename(columns={'exemplo':'PT/BR', 'libras':'LIBRAS' })[['id', 'ident', 'PT/BR', 'LIBRAS']]
df = df.drop_duplicates(subset=['PT/BR'])
df['ident'].nunique(), df['id'].nunique(), df.shape

(5804, 5804, (5804, 4))

In [5]:
# As soletradas eu juntei para ver se dá um resultado melhor no fewshot
def unir_hifens(texto):
    return re.sub(r'(\b[A-Z])(-[A-Z])+\b', lambda match: match.group(0).replace('-', ''), texto)
df['LIBRAS'] = [unir_hifens(i) for i in df['LIBRAS']]
df.head()

,id,ident,PT/BR,LIBRAS
0,1,7049,Invente qualquer palavra que comece com a letra A.,VOCÊ INVENTAR QUALQUER PALAVRA COMEÇAR A.
1,2,5402,Comprei um carro novo e paguei à vista.,EU COMPRAR CARRO NOVO PAGAR-À-VISTA.
2,3,6959,"Aquela mulher é muito sensível, chora à-toa.",MULHER AQUEL@ SINSÍVELmuito CHORAR À-TOA.
3,4,3930,Você gosta de abacate com leite?,VOCÊ GOSTAR ABACATE LEITE JUNTO?
4,5,5,"Hoje tomei suco de abacaxi, ele estava ácido.",HOJE SUCO ABACAXI BEBER ÁCID@.


In [13]:
df['tamanho'] = df['LIBRAS'].apply(lambda x: len(x.split()))
df['numerais'] = df['LIBRAS'].str.contains(r'\d')
df['contain_simbolos'] = df['LIBRAS'].str.contains(r'[@&^]')
df['interrogativa_L'] = df['LIBRAS'].str.contains(r'[?]')
df['exclamativa_L'] = df['LIBRAS'].str.contains(r'[!]')
df['afirmativa_L'] = df['LIBRAS'].str.contains(r'[.]')
df['travessao_L'] = df['LIBRAS'].str.contains(r'[-]')
df['interrogativa_P'] = df['PT/BR'].str.contains(r'[?]')
df['exclamativa_P'] = df['PT/BR'].str.contains(r'[!]')
df['afirmativa_P'] = df['PT/BR'].str.contains(r'[.]')

condicionais_libras = (df['contain_simbolos'] == False) & (df['numerais'] == False)

df[(df['afirmativa_P'] == True) & condicionais_libras][['PT/BR', 'LIBRAS', 'tamanho']].sort_values(by='tamanho', ascending=False)[:6]

,PT/BR,LIBRAS,tamanho
4543,"Antes eu ganhava pouco, só recebia bolsa de estudo; agora prosperei, ganho bem.",EU ANTES GANHAR POUCO SÓ RECEBER DINHEIRO BOLSA AGORA EU PROSPERAR GANHAR BEM.,13
4506,"No princípio eu não sabia como fazer o dicionário no computador, agora já acostumei.",PRINCÍPIO EU SABER NADA COMO FAZER DICIONÁRIO COMPUTADOR AGORA JÁ ACOSTUMAR.,11
131,"Aqui no Brasil transforma-se a cana em açúcar. Lá na Europa é diferente, transforma-se a beterraba em açúcar.",BRASIL AQUI CANA TRANSFORMAR AÇÚCAR LÁ EUROPA DIFERENTE BETERRABA TRANSFORMAR AÇÚCAR.,11
3254,"Os animais e os bebês precisam tomar leite, é bom para ajudar contra as doenças.",QUALQUER ANIMAL OU BEBÊ PRECISAR TOMAR LEITE BOM AJUDAR CONTRA DOENÇA.,11
4593,"Puxei a gaveta e de repente ela caiu, bateu no meu pé e eu gritei com muita dor.",GAVETA EU PUXAR-GAVETA DE-REPENTE GAVETA coisa-planaCAIR BATER PÉ EU GRITAR DOR.,11
5394,"Aqui no Paraná, em julho, sempre há toró. Prefiro viajar para Fortaleza, onde sempre há sol.",PARANÁ JULHO SEMPRE TORÓ EU PREFERIR VIAJAR FORTALEZA LÁ SEMPRE SOL.,11


In [17]:
df[(df['interrogativa_P'] == True) & condicionais_libras][['PT/BR', 'LIBRAS', 'tamanho']].sort_values(by='tamanho', ascending=False)[10:15]

,PT/BR,LIBRAS,tamanho
3477,Qual você prefere com o pão: manteiga ou queijo?,PÃO VOCÊ PREFERIR MANTEIGA OU QUEIJO QUAL?,7
5254,"Cadê a tampa da panela? Por favor, procure, preciso dela.",PANELA TAMPA CADÊ? POR-FAVOR PROCURAR EU PRECISAR.,7
3594,"Você quer refrigerante? Não beba tudo, só a metade.",QUERER REFRIGERANTE? BEBER TUDO NÃO SÓ METADE.,7
4970,O sabor desse suco parece de tangerina ou laranja. Qual é?,SUCO SABOR PARECER TANGERINA OU LARANJA QUAL?,7
511,Você irá arbitrar o jogo do Flamengo e Vasco no Maracanã ?,VOCÊ ARBITRAR VAI JOGO FLAMENGO VASCO MARACANÃ?,7


In [18]:
df[(df['exclamativa_P'] == True) & condicionais_libras][['PT/BR', 'LIBRAS', 'tamanho']].sort_values(by='tamanho', ascending=False)[:6]

,PT/BR,LIBRAS,tamanho
4024,Ontem fui à padaria comprar pão e levei um susto com o aumento do preço!,ONTEM EU IR PADARIA COMPRAR PÃO EU SUSTO PREÇO AUMENTAR.,10
4865,"É melhor retirar um pouco o que você escreveu, eu não gostei!",VOCÊ ESCREVER EU NÃO-GOSTAR MELHOR VOCÊ RETIRAR POUCO.,8
4135,O carro entrou na São Paulo-Santos e paguei o valor do pedágio com aumento!,SÃO-PAULO SANTOS CARRO ENTRAR PAGAR PEDÁGIO VALOR AUMENTAR!,8
2937,Você vai comprar carro!? Impossível! Onde irá conseguir dinheiro?,VOCÊ VAI COMPRAR CARRO!? IMPOSSÍVEL! ONDE CONSEGUIR DINHEIRO?,8
4679,"No Rio de Janeiro há poucos rabinos, em São Paulo há mais!",RABINO RIO TER POUCO SÃO-PAULO TER MAIS!,7
283,"Ontem, eu estava com enxaqueca, tomei comprimido, a cabeça melhorou e aliviou!",ONTEM EU ENXAQUECA TOMAR-COMPRIMIDO CABEÇA MELHOR ALIVIAR!,7
